In [50]:
import os

from utils import EXP_ROOT, load_jsonl

lang = "ja"
qa_root = os.path.join(EXP_ROOT, "datasets/kg-datasets/ja-0.5/eval_qa/03_en_qa")
ner_root = os.path.join(EXP_ROOT, "datasets/kg-datasets/ja-0.5/eval_qa/01_fact_check")

if lang == "ja":
    response_path = os.path.join(qa_root, "ja_generation.jsonl")
    ner_path = os.path.join(ner_root, "bionlp_ners.jsonl")
else:
    response_path = os.path.join(qa_root, "en_generation.jsonl")
    ner_path = os.path.join(ner_root, "ja_ners.jsonl")

responses = load_jsonl(response_path)
en_docs = {}
for item in load_jsonl(ner_path):
    en_docs[item['docid']] = item['abstract']

for item in responses:
    if "target_lang_abs" not in item['metadata']:
        assert item['metadata']['docid'] in en_docs, f"Missing docid {item['docid']} in en_docs"
        item['metadata']['target_lang_abs'] = en_docs[item['metadata']['docid']]

In [51]:
request_ids = [item['id'] for item in load_jsonl(f"../99_adaxeval_datasets/{lang}_knowledge_memorization.jsonl")]
print(f"Total {len(request_ids)} requests, {len(responses)} responses")
responses = [item for item in responses if item['request_id'] in request_ids]
print(f"Filtered to {len(responses)} responses")

Total 2553 requests, 3934 responses
Filtered to 2553 responses


In [52]:
import random 

def get_responses(method="random", preload_file=None):
    assert method in ["random", "preloaded"], "Method must be 'random' or 'preloaded'"
    if method == "random":   
        random.seed(42)
        random.shuffle(responses)
        for idx in range(100):
            yield responses[idx]
    else:
        import pandas as pd
        assert preload_file is not None, "Preload file must be specified for 'preloaded' method"
        assert os.path.exists(preload_file), f"Preload file {preload_file} does not exist"
        loaded_responses = pd.read_csv(preload_file)
        request_ids = loaded_responses["request_id"].tolist()

        id2response = {response["request_id"]: response for response in responses}
        for request_id in request_ids:
            if request_id in id2response:
                yield id2response[request_id]
            else:
                raise ValueError(f"Request ID {request_id} not found in responses")


In [ ]:
import csv
import random

# Define the output CSV file
output_path = f"./{lang}_evaluation.csv"

if os.path.exists(output_path):
    raise ValueError(f"Output file {output_path} already exists. Please remove it before running the script.")

# Define the header
header = [
    "request_id", "sentence", "triple", "prompt", "answer", 
    "faithfulness", 
    "paraphrase", "fluency", "factuality",  "diversity", "inter-factuality",
    "distractor_1", "d1_plausibility", "d1_incorrectness",
    "distractor_2", "d2_plausibility", "d2_incorrectness",
    "distractor_3", "d3_plausibility", "d3_incorrectness"
]

faithfulness_cnt = 0 

# Open CSV for writing
with open(output_path, mode="w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=header)
    writer.writeheader()

    for response in get_responses(method="random"):
        request_id = response['request_id']
        generation = response["generation"]
        triple = generation['triple']
        prompt = generation["fill_in_blank"]
        paraphrase = generation["question"]
        distractors = generation["distractors"]
        
        correct_ans = generation["answer"]
        sentence = response['metadata']['sentence']

        # Shuffle for display, but keep distractors labeled
        all_options = [correct_ans] + distractors
        random.shuffle(all_options)

        if prompt.replace("[BLANK]", correct_ans) in sentence:
            faithfulness = "3"
            faithfulness_cnt += 1
        else: 
            faithfulness = ""
        row = {
            "request_id": request_id,
            "sentence": sentence.replace("\n", " "),
            "triple": str(triple),
            "prompt": prompt,
            "answer": correct_ans,
            "faithfulness": faithfulness, 
            "paraphrase": paraphrase,
            "fluency": "",
            "factuality": "",
            "diversity": "",
            "inter-factuality": "",
            "distractor_1": distractors[0],
            "d1_plausibility": "", "d1_incorrectness": "",
            "distractor_2": distractors[1],
            "d2_plausibility": "","d2_incorrectness": "",
            "distractor_3": distractors[2],
            "d3_plausibility": "","d3_incorrectness": "",
        }

        writer.writerow(row)
print(faithfulness_cnt)
print(f"✅ CSV file written to {output_path}")

10
✅ CSV file written to ./ja_evaluation.csv


In [53]:
import random


for idx, response in enumerate(get_responses(method="preloaded", preload_file=f"./{lang}_evaluation.csv")):
    correct_ans = response['generation']['answer']
    options = [response['metadata']['input']['triple']['object']]
    options += response["generation"]["distractors"]
    sentence = response['metadata']['sentence']
    triple = response['metadata']['input']['triple']
    random.shuffle(options)
    
    print(f"\n=== {idx + 1} ===")
    print("### 输入")
    print("Original sentence: ", sentence)
    # print("Extracted Triple: ", triple)
    print("BLANK prompt: ", response["generation"]["fill_in_blank"])
    print("Question: ", response["generation"]["question"])
    print(f"Correct answer: {correct_ans}\n")
    # print(f"1. {options[0]}\n2. {options[1]}\n3. {options[2]}\n4. {options[3]}")
    
    # print(f"Japanese abstract: ", response['metadata']['target_lang_abs'])
    # print("English Triple: ", triple)
    # print(f"Japanese abstract: ", response['metadata']['abstract'])


=== 1 ===
### 输入
Original sentence:  破裂性腹部大動脈瘤(rAAA)に対する腹部大動脈ステントグラフト内挿術(EVAR)術後の重要な合併症の1つに腹部コンパートメント症候群(ACS)がある.
BLANK prompt:  破裂性腹部大動脈瘤(rAAA)に対する腹部大動脈ステントグラフト内挿術(EVAR)術後の重要な合併症の1つに[BLANK]がある。
Question:  破裂性腹部大動脈瘤(rAAA)に対するEVAR術後に重要な合併症として挙げられる疾患は何ですか？
Correct answer: 腹部コンパートメント症候群（ACS）


=== 2 ===
### 输入
Original sentence:  PEHは気管支鏡やCTガイド下肺生検では確定診断を得ることは困難であり,外科的肺生検が診断に有用となることが多いと考える.
BLANK prompt:  PEHは気管支鏡やCTガイド下肺生検では確定診断を得ることは困難であり、[BLANK]が診断に有用となることが多いと考える。
Question:  PEHの確定診断を得るために有用とされる検査方法は何ですか？
Correct answer: 外科的肺生検


=== 3 ===
### 输入
Original sentence:  スキンシーラントは皮膚用微生物密封材で,消毒後も残存した皮膚菌叢を物理的に封じ込め,菌の移動と発現を阻止し手術部位感染を防ぐ効果がある.
BLANK prompt:  [BLANK]は皮膚用微生物密封材で、消毒後も残存した皮膚菌叢を物理的に封じ込め、菌の移動と発現を阻止し手術部位感染を防ぐ効果がある。
Question:  消毒後も残存した皮膚菌叢を物理的に封じ込め、菌の移動と発現を阻止し手術部位感染を防ぐ効果がある皮膚用微生物密封材は何ですか？
Correct answer: スキンシーラント


=== 4 ===
### 输入
Original sentence:  神経線維の膨大した終末と褐色脂肪細胞および毛細血管との間にシナップスが形成される.
BLANK prompt:  神経線維の膨大した終末と[BLANK]との間にシナップスが形成される。
Question:  神経線維の膨大した終末がシナプスを形成する相手として挙げられる細

In [34]:
for idx, response in enumerate(get_responses(method="preloaded", preload_file="./annotation/ja_evaluation.v2.csv")):
    triple = response['metadata']['input']['triple']
    sentence = response['metadata']['sentence']
    prompt = response["generation"]["prompt"]
    paraphrase = response["generation"]["question"]
    correct_ans = triple['object']
    distractors = response["generation"]["distractors"]
    
    print(f"\n=== {idx + 1} ===")
    print(f"sentence: {sentence}")
    print(f"triple: {triple}")
    print(f"question: {paraphrase}")
    print(f"distractor 1: {distractors[0]}")
    print(f"distractor 2: {distractors[1]}")
    print(f"distractor 3: {distractors[2]}")



=== 1 ===
sentence: 「絵具」は,着色剤(顔料)と展色剤の混合物であり,着色剤は,展色剤に溶けるものであってはならない。
triple: {'subject': '絵具', 'relation': 'は', 'object': '着色剤(顔料)と展色剤の混合物'}
question: 絵具はどのような成分の組み合わせからできていますか？
distractor 1: 展色剤のみからなる物質
distractor 2: 着色剤(顔料)と水の混合物
distractor 3: 顔料と溶剤の単純な混合物

=== 2 ===
sentence: オオシマザクラを含むバラ科の自家不和合性にはS-RNaseが関与し、S-RNaseのDNA多型に基づいて、個体のS遺伝子型が推定できる。
triple: {'subject': 'オオシマザクラを含むバラ科の自家不和合性', 'relation': 'に関与し', 'object': 'S-RNase'}
question: オオシマザクラを含むバラ科の自家不和合性に関与している分子は何ですか？
distractor 1: 花粉管伸長因子
distractor 2: アクチンフィラメント
distractor 3: 花粉外被タンパク質

=== 3 ===
sentence: シェーグレン症候群(SS)は,慢性唾液腺炎と乾燥性角結膜炎を主徴とする自己免疫疾患の一つである.
triple: {'subject': 'シェーグレン症候群(SS)', 'relation': 'は...主徴とする', 'object': '慢性唾液腺炎と乾燥性角結膜炎'}
question: シェーグレン症候群（SS）では、どのような症状が主な特徴とされていますか？
distractor 1: 慢性関節炎と皮膚発疹
distractor 2: 筋力低下と神経障害
distractor 3: 発熱とリンパ節腫脹

=== 4 ===
sentence: 実際、隕石中に含まれている鉱物は珪酸塩鉱物(かんらん石、輝石、長石)と鉄ニッケル合金が中心であり、シリカ鉱物は一部の隕石に少量報告されている程度である。
triple: {'subject': '隕石中に含まれる鉱物', 'relation': 'は...中心であり', 'object':